# Bedrock Implementation of Medium Analyzer

In this notebook we will use Claude for the LLM via Bedrock and the Titan Embeddings Model to build out a simple RAG Workflow orchestrated by LangChain.

## Credits/Additional References
- <b>Bedrock OSS RAG Original Implementation</b>: https://github.com/aws-samples/amazon-bedrock-workshop/blob/main/06_OpenSource_examples/01_Langchain_KnowledgeBases_and_RAG_examples/01_qa_w_rag_claude.ipynb
- <b>Bedrock Workshop Official Open Source Samples</b>: https://github.com/aws-samples/amazon-bedrock-workshop/tree/main/06_OpenSource_examples
- <b>Bedrock AWS Samples Repo</b>: https://github.com/aws-samples/amazon-bedrock-samples/tree/main

## Setup
Can use any Python environment that has Boto3 access to the Bedrock models, in this case we use a SageMaker classic notebook instance with an ml.c5.xlarge CPU instance. If working in a GPU environment ensure to install faiss-gpu.

In [ ]:
%pip install langchain-community faiss-cpu==1.8.0 langchain pypdf #restart kernel after installation

In [ ]:
import boto3
import botocore
import langchain
from langchain.embeddings.cache import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.document_loaders import PyPDFDirectoryLoader

boto3_bedrock = boto3.client('bedrock-runtime')

## Sample Boto3 Inference With Claude V2
Can update this Claude model to the 3.x family if needed, here's how a sample API call looks like via boto3.

In [ ]:
import json

model_id = 'anthropic.claude-v2'
accept = "application/json"
contentType = "application/json"

prompt_data = """Human: Write me a small paragraph saying nice things about me.

Assistant:
"""
print(prompt_data)

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
response = boto3_bedrock.invoke_model(
    body=body, modelId=model_id, accept=accept, contentType=contentType
)
response_body = json.loads(response.get("body").read())
print(response_body.get("completion"))

## Embeddings & Vector Store Setup
In this case for the RAG stack we use the following:

- <b>Embeddings Model</b>: Amazon Titan via Bedrock
- <b>Vector Store</b>: FAISS

Ensure that you have the sagemaker-articles directly cloned as well or replace this with your own set of PDF documents.

In [ ]:
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

# where our embeddings will be stored
store = LocalFileStore("./cache/")

# instantiate a loader: this loads our data, use PDF in this case
loader = PyPDFDirectoryLoader("sagemaker-articles/")

# by default the PDF loader both loads and splits the documents for us
pages = loader.load_and_split()
print(len(pages))

# create the LLM and Embeddings Models object via LangChain
llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':200})
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=boto3_bedrock)

# pass in our vector store
embedder = CacheBackedEmbeddings.from_bytes_store(
    bedrock_embeddings,
    store
)

In [ ]:
# instantiate vector store, we use FAISS in this case
vector_store = FAISS.from_documents(pages, embedder)

## Chain Creation & Inference

We can wrap all these objects into a singular Retrieval QA chain (RAG): https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval_qa.base.RetrievalQA.html.

In [ ]:
# Promot Template for the Retrieval QA chain
def fill_prompt(template, human_text):
    # Replace the placeholder 'Human:' with the provided human_text
    filled_prompt = template.replace("Human:", f"Human: {human_text}")
    return filled_prompt

# Claude structured template
prompt_data = """Human:

Assistant:
"""

# this is the entire retrieval system
from langchain.chains import RetrievalQA
medium_qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
    verbose=True,
)

## Sample Inference with RAG and Vanilla Bedrock Model

In [ ]:
sample_prompts = ["What does Ram Vegiraju write about?",
                 "What is Amazon SageMaker?",
                 "What is Amazon SageMaker Inference?",
                 "What are the different hosting options for Amazon SageMaker?",
                 "What is Serverless Inference with Amazon SageMaker?",
                 "What's the difference between Multi-Model Endpoints and Multi-Container Endpoints?",
                 "What SDKs can I use to work with Amazon SageMaker?"]

In [ ]:
for prompt in sample_prompts:
    print(prompt)
    print("------------------------------------")
    print("Vanilla Bedrock Response")
    print("------------------------------------")
    prompt_template = fill_prompt(prompt_data, prompt)
    body = json.dumps({"prompt": prompt_template, "max_tokens_to_sample": 500})
    response = boto3_bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())
    print(response_body.get("completion"))
    print("------------------------------------")
    print("RAG Enabled Response")
    print("------------------------------------")
    response_rag = medium_qa_chain({"query":prompt})
    print(response_rag['result'])